In [1]:
pip install pandas sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd



In [24]:


companies_df = pd.read_csv('C:/Users/secar/OneDrive/Documents/Bootcamp/Project4/sp500_companies.csv')
stock_df = pd.read_csv('C:/Users/secar/OneDrive/Documents/Bootcamp/Project4/sp500_stocks.csv')

stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df = stock_df[(stock_df['Date'] >= '2010-01-01') & (stock_df['Date'] <= '2013-12-31')]


In [25]:
from sqlalchemy import create_engine
#create/connect to engine
engine = create_engine('sqlite:///:memory:')

In [26]:
# Store the DataFrames in the SQLite database
companies_df.to_sql('table1', con=engine, index=False, if_exists='replace')
stock_df.to_sql('table2', con=engine, index=False, if_exists='replace')

506018

In [27]:
from sqlalchemy import MetaData, Table, select, outerjoin
from sqlalchemy.orm import aliased

metadata = MetaData()

# Reflect tables from the database
table1 = Table('table1', metadata, autoload_with=engine)
table2 = Table('table2', metadata, autoload_with=engine)

# Perform full outer join
t1 = aliased(table1)
t2 = aliased(table2)

stmt1 = select(t1, t2).outerjoin(t2, t1.c.Symbol == t2.c.Symbol)
stmt2 = select(t1, t2).outerjoin(t1, t2.c.Symbol == t1.c.Symbol)
union_query = stmt1.union_all(stmt2)

with engine.connect() as connection:
    result = connection.execute(union_query)
    merged_df = pd.DataFrame(result.fetchall(), columns=result.keys())



In [28]:
# Export the merged DataFrame to an Excel file
excel_output_path = "merged_stocks.xlsx"
merged_df.to_excel(excel_output_path, index=False)
